how to solve this thing?

I need to do the process of a nural net. It's not that complicated but requiers a bit of finageling.

We start with splitting the data into train and validate sets.

Then we need to create the nural net that will run the calculations.

We will copy the pytorch interface

you have a Modle class, this encapuslates everything



In [ ]:
import argparse
from typing import Optional
from abc import ABC, abstractmethod

import numpy as np
from numpy.typing import NDArray

import common as cm

Array = NDArray[np.float32]

In [ ]:
# Abstract class to reprisent a layer in the nural net
class Layer(ABC):
    @abstractmethod
    def forward(self, x: Array) -> Array:
        pass

    @abstractmethod
    def backward(self, grad_output: Array) -> Array:
        pass

# Linear transform for incomming data
#   y = x @ W + b
#
# 
# x = input   shape(num_rows, in_features)
# W = weights shape(in_features, out_features)
# b = bias    shape(out_features,)
# y = output  shape(num_rows, out_features)
#
# ^1 the @ is matrix multiply same simbole is used in numpy
#
class Linear(Layer):
    def __init__(self, in_features: int, out_features: int) -> None:

        # Init weights using Xavier Initialization
        limit = 1 / np.sqrt(in_features)
        self.W: Array = np.random.uniform(-limit, limit, (in_features, out_features)).astype(np.float32)
        
        # Init bias to zero
        self.b: Array = np.zeros(out_features, dtype=np.float32)

        # gradients
        self.dW: Array = np.zeros_like(self.W)
        self.db: Array = np.zeros_like(self.b)

        self.x: Array | None = None  # cache input for backward pass

    def forward(self, x: Array) -> Array:
        self.x = x
        return x @ self.W + self.b

    def backward(self, grad_output: Array) -> Array:
        assert self.x is not None
        # gradients are stored so the optimiser can read them
        # later to adjust them using the learing reate
        self.dW = self.x.T @ grad_output  # gradient of loss with respect to (wrt) weights 
        self.db = grad_output.sum(axis=0) # gradient of loss wrt bias
        # gradient wrt input
        return grad_output @ self.W.T

class ReLU(Layer):
    def __init__(self) -> None:
        self.mask: NDArray[np.bool] | None = None
    
    def forward(self, x: Array) -> Array:
        self.mask = x > 0 # mask for gradients above zero, 
        return np.maximum(x, 0)

